## IMPORTS

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import root_mean_squared_error, r2_score
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import GridSearchCV

In [ ]:
df = pd.read_csv('train.csv')
df.dropna(subset=['value_eur'], inplace=True)
df = df[~df["player_positions"].str.contains("GK", na=False)]

In [ ]:
df.info()

## DATA PREPROCESSING

In [ ]:
# drop irrelevant columns
cols_to_drop = ['Unnamed: 0', 'id', 'short_name', "long_name", "dob", "club_jersey_number", "nation_jersey_number", "club_id", "club_loaned_from", "nation_position", "player_traits", "player_tags", "nationality_name", "club_team_id", "nationality_id"]

df.drop(columns=cols_to_drop, inplace=True, errors='ignore')

In [ ]:
# contract related variables manipulation
start_date = float(2021) # year when dataset was recorded 
df["years_left_contract"] = df["club_contract_valid_until"] - start_date

df['club_joined'] = pd.to_datetime(df['club_joined'], format='%Y-%m-%d')
df["years_in_club"] = start_date - df["club_joined"].dt.year

df.drop(columns=["club_contract_valid_until", "club_joined"], inplace=True, errors='ignore')

In [ ]:
# work_rate split 
df[["work_attack", "work_defense"]] = df["work_rate"].str.split("/", expand=True)
maps = {"Low": 1, "Medium": 2, "High": 3}
df["work_attack"] = df["work_attack"].map(maps)
df["work_defense"] = df["work_defense"].map(maps)

In [ ]:
# target based ordinal encoding for positions
df[["primary_position", "secondary_position", "tertiary_position"]] = df["player_positions"].str.split(",", expand=True)

club_position_map = df.groupby('club_position')['value_eur'].median().sort_values().rank().to_dict()
club_position_map[None] = 0
df["club_position"] = df["club_position"].map(club_position_map)
df["primary_position"] = df["primary_position"].map(club_position_map)
df["secondary_position"] = df["secondary_position"].map(club_position_map)
df["tertiary_position"] = df["tertiary_position"].map(club_position_map)

df["is_multiposition"] = df["player_positions"].apply(lambda x: len(x.split(",")) > 1).astype(int)
df.drop(columns=["player_positions"], inplace=True, errors='ignore')

In [ ]:
# Starter Bool
df["is_starter"] = (~df["club_position"].isin(["SUB", "RES"])).astype(int)

In [ ]:
cat_cols = df.select_dtypes(include=['object']).columns

for col in cat_cols:
    df[col] = df[col].astype('category')

In [ ]:
# data engineering
df["overall_potential"] = df["overall"] * df["potential"]

In [ ]:
# target based ordinal enconding for club and league
club_map = df.groupby('club_name', observed=True)['value_eur'].median().sort_values().rank().to_dict()
df["club_name"] = df["club_name"].map(club_map)

league_map = df.groupby('league_name', observed=True)['value_eur'].median().sort_values().rank().to_dict()
df["league_name"] = df["league_name"].map(league_map)


In [ ]:
# target based ordinal encoding for work rate
work_rate_map = df.groupby('work_rate', observed=True)['value_eur'].median().sort_values().rank().to_dict()
df["work_rate"] = df["work_rate"].map(work_rate_map)

## TRAIN

In [ ]:
X = df.drop(columns=["value_eur"], errors='ignore')
y = df["value_eur"]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [ ]:
# List of columns to keep
cols_to_keep = [
    "release_clause_eur", "overall_potential", 
    "wage_eur", "age", "league_name", 
    "club_name", "work_rate", "is_starter",
    "secondary_position", "tertiary_position",
]

# Filter the DataFrame to keep only the specified columns
X_train = X_train[cols_to_keep]
X_test = X_test[cols_to_keep]

In [ ]:
rf = RandomForestRegressor(
    n_estimators=100,
    max_depth=100,
    random_state=42,
)

rf.fit(X_train, y_train)
train_preds = rf.predict(X_train)
test_preds = rf.predict(X_test)

# Evaluate the model
train_rmse = root_mean_squared_error(y_train, train_preds)
test_rmse = root_mean_squared_error(y_test, test_preds)
print(f"Train RMSE: {train_rmse}")
print(f"Test RMSE: {test_rmse}")
print(f"Train R^2: {r2_score(y_train, train_preds)}")
print(f"Test R^2: {r2_score(y_test, test_preds)}")